In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from hanspell import spell_checker
from selenium import webdriver
from BoxScoreParser import BoxScoreParser
from UrlParser import UrlParser
from sklearn import tree
from sklearn.externals.six import StringIO
from selenium import webdriver

import pandas as pd
import numpy as np
import MySQLdb
import random
import datetime
import urllib2
import os
import sys

In [2]:
# DecisionTree - 서두
path_INTRO = './DecisionLearning//DecisionTree_sample.csv'
df_INTRO = pd.read_csv(path_INTRO,index_col=None,header=None,names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum','context'])
score_INTRO = np.array(df_INTRO.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context_INTRO = np.array(df_INTRO.get('context'))
context_sen_INTRO = [' ','승리하였다.','접전끝에 승리를 하였다.','역전에 성공했다.','완승을 하였다.','무승부로 끝이났다.','영봉승을 하였다.']
clf_INTRO = tree.DecisionTreeClassifier(criterion='entropy')
clf_INTRO = clf_INTRO.fit(score_INTRO,context_INTRO)

# DecisionTree - 투수
path_MVP_PITCHER = './DecisionLearning/MVP_Pitcher_sample.csv'
df_MVP_PITCHER = pd.read_csv(path_MVP_PITCHER,index_col=None,header=None,names=['ERA','BN','H','SO','context'])
score_MVP_PITCHER = np.array(df_MVP_PITCHER.reindex(columns=['ERA','BN','H','SO']))
context_MVP_PITCHER = np.array(df_MVP_PITCHER.get('context'))
clf_MVP_PITCHER = tree.DecisionTreeClassifier(criterion='entropy')
clf_MVP_PITCHER = clf_MVP_PITCHER.fit(score_MVP_PITCHER,context_MVP_PITCHER)

In [3]:
def writeDate(url):    
    date=bxsParser.date[4:6]+u"월"+bxsParser.date[6:8]+u"일"+bxsParser.date[8:]
    return date

In [4]:
def writeHead(url,contextClassifier,bxsParser):

    # HEAD
    HEAD=u'at ascore vs ht hscore'
    HEAD=HEAD.replace('at',bxsParser.a_team)
    HEAD=HEAD.replace('ht',bxsParser.h_team)
    HEAD=HEAD.replace('hscore',str(bxsParser.h_score[12]))
    HEAD=HEAD.replace('ascore',str(bxsParser.a_score[12]))
    
    return HEAD

In [5]:
def getClassifier(url):
    return clf_INTRO.predict([x-y for x,y in zip(bxsParser.h_score[:13],bxsParser.a_score[:13])])[0]

In [6]:
def writeIntro(url,contextClassifier,bxsParser):
    MVP_PITCHER=None
    
    sentences1=(
        u'winTeam는 date stadium 경기장에서 열린 loseTeam와의 경기에서 ascore-hscore 으로 context',
        u'date stadium에서 열린 at와 ht와의 경기에서 ascore-hscore로 winTeam이 context',
        u'at와 ht의 경기가 date stadium에서 있었던 오늘 at는 ascore, ht는 hscore로 winTeam가 경기에서 context',
        u'date stadium에서 있었던 at와 ht와의 경기, winTeam이 winScore loseTeam이 loseScore로 winTeam이 context',
    )
    sentences2=(
        u'이날 경기에서 mvp_pitcher이 투구수NP 피안타H 의 기록을 보여주며 호투를 하였다.',
        u'이번 경기에서 winTeam의  mvp_pitcher이 INN이닝을 호투하며, R실점 SO삼진을 잡아내며 승리를 이끌었다.'
    )
    
    # INTRO1
    INTRO1=sentences1[random.randint(0,len(sentences1)-1)]
    INTRO1=INTRO1.replace('winTeam',bxsParser.winTeam)
    INTRO1=INTRO1.replace('loseTeam',bxsParser.loseTeam)
    INTRO1=INTRO1.replace('winScore',str(bxsParser.winScore))
    INTRO1=INTRO1.replace('loseScore',str(bxsParser.loseScore))
    INTRO1=INTRO1.replace('stadium',bxsParser.stadium)
    INTRO1=INTRO1.replace('hscore',str(bxsParser.h_score[12]))
    INTRO1=INTRO1.replace('ascore',str(bxsParser.a_score[12]))
    INTRO1=INTRO1.replace('date',bxsParser.date)
    INTRO1=INTRO1.replace('ht',bxsParser.h_team)
    INTRO1=INTRO1.replace('at',bxsParser.a_team)    
    
    # INTRO-context
    #'context' -> exception_case(무승부)
    if bxsParser.h_score[12]==bxsParser.a_score[12]:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[5].decode('utf-8'))    
    #'context' -> exception_case(영봉승)
    elif bxsParser.h_score[12]==0 or bxsParser.a_score[12]==0:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[6].decode('utf-8'))    
    #'context'-> 나머지의 경우
    else:
        INTRO1=INTRO1.replace('context',context_sen_INTRO[getClassifier(url)].decode('utf-8'))
        
#     #HOME팀이 이긴 경우 승리->패배
#     if bxsParser.a_score[12] < bxsParser.h_score[12]:
#         INTRO1=INTRO1.replace(u'승리',u'패배')
#         INTRO1=INTRO1.replace(u'성공',u'실패')
#         INTRO1=INTRO1.replace(u'완승',u'완패')
#         INTRO1=INTRO1.replace(u'영봉승',u'영봉패')

#     이날 경기를 승리로 이끈 MVP_PITCHER는 +
#     date.encode('utf8')
#     head.encode('utf8')
#     pre.encode('utf8')
#     url.encode('utf8')
    
    # MVP_PITCHER
    for row in range(len(bxsParser.winTeam_pitRecord)):
        ERA=bxsParser.winTeam_pitRecord[u'평균자책점'][row]
        if type(ERA)==float:
            ERA=float(ERA)
        else:
            ERA=0
        BN=int(bxsParser.winTeam_pitRecord[u'타자'][row])
        H=int(bxsParser.winTeam_pitRecord[u'피안타'][row])
        SO=int(bxsParser.winTeam_pitRecord[u'삼진'][row])
        arr=np.array([ERA,BN,H,SO])
                       
        INTRO2=''
        if clf_MVP_PITCHER.predict(arr)[0]:
            MVP_PITCHER=bxsParser.winTeam_pitRecord[u'선수명'][row] 
            #INTRO2
            INTRO2=sentences2[random.randint(0,len(sentences2)-1)]
            INTRO2=INTRO2.replace('winTeam',bxsParser.winTeam)
            INTRO2=INTRO2.replace('mvp_pitcher',bxsParser.winTeam_pitRecord.iloc[row][u'선수명'])
            INTRO2=INTRO2.replace('ERA',bxsParser.winTeam_pitRecord.iloc[row][u'평균자책점'])
            INTRO2=INTRO2.replace('INN',bxsParser.winTeam_pitRecord.iloc[row][u'이닝'])
            INTRO2=INTRO2.replace('BN',bxsParser.winTeam_pitRecord.iloc[row][u'타자'])
            INTRO2=INTRO2.replace('NP',bxsParser.winTeam_pitRecord.iloc[row][u'투구수'])
            INTRO2=INTRO2.replace('SO',bxsParser.winTeam_pitRecord.iloc[row][u'삼진'])            
            INTRO2=INTRO2.replace('ER',bxsParser.winTeam_pitRecord.iloc[row][u'자책'])
            INTRO2=INTRO2.replace('R',bxsParser.winTeam_pitRecord.iloc[row][u'실점'])
            INTRO2=INTRO2.replace('H',bxsParser.winTeam_pitRecord.iloc[row][u'피안타'])
            break
            
    return INTRO1+INTRO2

In [7]:
# 결론작성
def writeConc(url,contextClassifier,bxsParser):
    pass

In [8]:
def getEmblem(bxsParser):
    if bxsParser.winTeam=='삼성':
        emblem='SS'
    elif bxsParser.winTeam=='한화':
        emblem='HH'
    elif bxsParser.winTeam=='SK':
        emblem='SK'
    elif bxsParser.winTeam=='KIA':
        emblem='HT'
    elif bxsParser.winTeam=='LG':
        emblem='LG'
    elif bxsParser.winTeam=='두산':
        emblem='OB'
    elif bxsParser.winTeam=='롯데':
        emblem='LT'
    elif bxsParser.winTeam=='넥센':
        emblem='WO'
    elif bxsParser.winTeam=='kt':
        emblem='KT'
    else:
        emblem='NC'
    return emblem

In [9]:
 def getCriticalInning(date,awayTeam):
#         print date,awayTeam
        critical_inning=[]
        if os.name=='nt':
            browser = webdriver.Chrome(executable_path='./chromedriver.exe')
        else:
            browser = webdriver.Chrome(executable_path='./chromedriver')

        year=date[0:4]        
        month=date[4:6]        
        day=int(date[6:8])
        dayCount=datetime.date.today()-datetime.date(int(year),int(month),day)
        dayCount=dayCount.days
        
        url='http://score.sports.media.daum.net/schedule/baseball/kbo/main.daum?game_year=%s&game_month=%s'%(year,month)
        page=urllib2.urlopen(url)
        html=BeautifulSoup(page)

        s=unicode(html)
        date_index=s.find(str.format('<td class="time_date" rowspan="5">%d<span class="txt_day">')%(day))
        s=s[date_index:]
        
        s=s[s.find(awayTeam):]
        serial_find_url='http://sports.media.daum.net/sports/gamecenter/'
        serial_start_index=s.find(serial_find_url)+len(serial_find_url)
        serial=s[serial_start_index:serial_start_index+8]
        
        url=str.format('http://m.sports.media.daum.net/m/sports/pack/3min/%s?lineup')%(serial)
        
        browser.get(url)
        soup=BeautifulSoup(browser.page_source)
        
        awayTeam = soup.select('div.awayth:nth-child(1) > div')
        sel=soup.select('strong.img_highlight')

        for inn in sel:
            try:
                inningNumber=inn.parent.parent.parent.attrs['data-inning']
                topbottom=inn.parent.parent.parent.attrs['data-half']
#                 inningNumber가 11이상이면 KBO사이트에서 흐름에 조회가 되지않음
                if int(inningNumber)<11:
                    critical_inning.append((inningNumber,topbottom))
            except :
                pass
        browser.close()
        return critical_inning

In [10]:
# 본문작성
def writeMain(url,contextClassifier,bxsParser):
    url=url.replace('BoxScore','Situation')
    month=bxsParser.date[4:6]
    day=bxsParser.date[6:8]
    critical_Inning=getCriticalInning(date='2016'+month+day,awayTeam=unicode(bxsParser.a_team))
#     MAIN1 : about critical Inning
    MAIN1=u''
#     MAIN2 : about the flow of critical Inning
    MAIN2=u''
#     MAIN3 : about the keyPlayer(batter)
    MAIN3=u''
    
    
#     MAIN1 BEGIN
    sen=u''
#     no critical Inning
    if len(critical_Inning)==0:
        sen+=u'결정적인 이닝이 없던 무난한 경기였다.'
#         1 or more critical Exist
    else:
        inning=','.join([inn[0] for inn in critical_Inning])
        sen+=u'승부는 inning이닝에서 갈리게 되었다.'
        sen=sen.replace('inning',inning)
#         MAIN1 END
    MAIN1+=sen

#     MAIN2 BEGIN
    for inn in critical_Inning:
#         open All Of Flow
        data=urllib2.urlopen(url+'#sms11')
        html=BeautifulSoup(data)
        cri=str.format('div.situation div#sms%02d > table'%(int(inn[0])))
#                              first = AWAY WIN
        if inn[1]=='first':
            cri=html.select(cri)[0]
#                              second= HOME WIN
        else:
            cri=html.select(cri)[1]
        
        hitContextList=[u'안타',u'루타',u'홈런']
        tmphitList=[]
        hitList=[]
        for line in cri.text.splitlines():
            line = line.strip()
            if line!='':
                tmphitList.append(line)
        for sen in tmphitList[2:]:
            hitList.append(sen.split(' ',2)[2])
            
        for sen in hitList:
            for context in hitContextList:
                if context in sen:
                    player=unicode(sen.split(' ',1)[0])
                    action=unicode(sen.split(' ',1)[1])
                    ex_sen=u'player의 action,'
                    ex_sen=ex_sen.replace(u'player',player) 
                    ex_sen=ex_sen.replace(u'action',action)
                    MAIN2+=ex_sen
                    
        MAIN2+=u'등이 bxsParser.winTeam의 승리에 큰 기여를 하였다.'
        MAIN2=MAIN2.replace(u'bxsParser.winTeam',bxsParser.winTeam)
#         MAIN2 END

#         MAIN3 BEGIN
    MAIN3=u'한편, '
    data = urllib2.urlopen('http://m.sports.media.daum.net/m/sports/pack/3min/71041552?lineup')
    year=bxsParser.date[:4]
    month=bxsParser.date[4:6]
    day=bxsParser.date[6:8]
    html=BeautifulSoup(data)
    
    if os.name=='nt':
        browser = webdriver.Chrome(executable_path='./chromedriver.exe')
    else:
        browser = webdriver.Chrome(executable_path='./chromedriver')
    
    
    url='http://score.sports.media.daum.net/schedule/baseball/kbo/main.daum?game_year=%s&game_month=%s'%(year,month)
    page=urllib2.urlopen(url)
    html=BeautifulSoup(page)
    
    s=unicode(html)
    date_index=s.find(str.format('<td class="time_date" rowspan="5">%d<span class="txt_day">')%(int(day)))
    s=s[date_index:]
    
    s=s[s.find(bxsParser.a_team):]
    serial_find_url='http://sports.media.daum.net/sports/gamecenter/'
    serial_start_index=s.find(serial_find_url)+len(serial_find_url)
    serial=s[serial_start_index:serial_start_index+8]
    
    url=str.format('http://m.sports.media.daum.net/m/sports/pack/3min/%s?lineup')%(serial)
    
    browser.get(url)
    soup=BeautifulSoup(browser.page_source)
    if bxsParser.a_scoreSum > bxsParser.h_scoreSum:
        sel=soup.select_one('td.position.away.key-player')
    else:
        sel=soup.select_one('td.position.home.key-player')
    
    keyPlayer=sel.findNextSibling().text
    keyPlayerBattingAverage=sel.findNextSibling().findNextSibling().text.lstrip()[:4]
    keyPlayerBattingInfo_toDict=bxsParser.winTeam_batRecord[bxsParser.winTeam_batRecord[u'선수명']==keyPlayer]
    keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
    
    keyPlayerMovement=[key+' '+keyPlayerBattingInfo_toDict[key] for key in [u'득점',u'타율',u'타점',u'타수',u'안타']]
    
    MAIN3+=u'bxsParser.winTeam의 keyPlayer이 Deukjeom Tayul Tajeom Tasu Anta'
    MAIN3=MAIN3.replace(u'bxsParser.winTeam',bxsParser.winTeam)
    MAIN3=MAIN3.replace(u'keyPlayer',keyPlayer)
    MAIN3=MAIN3.replace(u'Deukjeom',u'득점 '+keyPlayerBattingInfo_toDict[u'득점'])
    MAIN3=MAIN3.replace(u'Tayul',u'타율 '+keyPlayerBattingInfo_toDict[u'타율'])
    MAIN3=MAIN3.replace(u'Tajeom',u'타점 '+keyPlayerBattingInfo_toDict[u'타점'])
    MAIN3=MAIN3.replace(u'Tasu',u'타수 '+keyPlayerBattingInfo_toDict[u'타수'])
    MAIN3=MAIN3.replace(u'Anta',u'안타 '+keyPlayerBattingInfo_toDict[u'안타'])
    
    MAIN3+=u'의 기록으로 키플레이어의 자리에 올랐다.'
    
    browser.close()
                
    return MAIN1+MAIN2+MAIN3

In [11]:
startDate='20160724'
endDate='20160701'
urlParser = UrlParser(startDate,endDate)

In [ ]:
# MySQL conf
con=MySQLdb.connect(host='218.150.181.131',user='root',passwd='1234',db='link10th',charset='utf8', use_unicode=True)

for url in urlParser.urlList:
    try:
        bxsParser=BoxScoreParser(url)
        date=writeDate(url)
        contextClassifier=getClassifier(url)
        Head=writeHead(url,contextClassifier,bxsParser)
        Intro=writeIntro(url,contextClassifier,bxsParser)
        Main=writeMain(url,contextClassifier,bxsParser)
    #     Conc=writeConc(url,contextClassifier,bxsParser)
        emblem=getEmblem(bxsParser)
    except:
        sys.stderr.write(unicode.format(u'********** 날짜 : %s %s와 %s의 경기 파싱오류 ********** '%(bxsParser.date,bxsParser.a_team,bxsParser.h_team)))
    
    #교정부분
    Head = spell_checker.check(Head)
    Intro = spell_checker.check(Intro)
    Main = spell_checker.check(Main)
#     Conc = spell_checker.check(Conc)
    Head = Head.checked
    Intro = Intro.checked
    Main = Main.checked
#     Conc = Conc.checked
     
    cursor=con.cursor()
    cursor.execute(
    '''
    INSERT INTO Article(date,Head,Intro,Main,url,emblem)
    VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')
    '''
    %(date,Head,Intro,Main,url,emblem))

    # MySQL disconnect
    con.commit()
    cursor.close()
con.close()

In [ ]:
con=MySQLdb.connect(host='218.150.181.131',user='root',passwd='1234',db='link10th',charset='utf8', use_unicode=True)
cursor=con.cursor()
cursor.execute(
'''
SET global max_allowed_packet=1000000;
'''
)
# MySQL disconnect
con.commit()
cursor.close()
con.close()

In [13]:
url='http://m.sports.media.daum.net/m/sports/pack/3min/71041578?pitcher'
driver=webdriver.PhantomJS(executable_path='./phantomjs')
driver.get(url)
driver.page_source

<!DOCTYPE html><html lang="ko"><head>
<meta charset="UTF-8">
<title>3분 야구</title>
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, width=device-width">
<meta name="keywords" content="스포츠뉴스,다음스포츠,스포츠">
<meta name="svcdomain" content="m.sports.media.daum.net">


		





																																																																																												

			<meta property="og:author" content="다음스포츠">
				<meta property="og:title" content="선발야구 이끄는 좌완 에이스">
				<meta property="og:description" content="4시간짜리 경기를 3분만에?! 야구 좀 아는 에디터가 엑기스만 뽑아 제대로 정리해 드립니다. 고화질 영상과 뉴스, 경기평점까지 한눈에 보는 고품격 리뷰 'Daum 3분야구'를 만나보세요.">
				<meta property="og:site_name" content="다음스포츠">
				<meta property="og:image" content="http://t1.daumcdn.net/section/oc/8bcd9ed542a547b08177123d89ad8309">
				<meta property="og:url" content="http://m.sports.media.daum.net/m/sports/pack/3min/71041578?pitcher">
	
<lin